In [4]:
class LL1ParsingTable:
    def __init__(self, non_terminals, terminals, start_symbol, productions):
        self.non_terminals = non_terminals
        self.terminals = terminals
        self.start_symbol = start_symbol
        self.productions = productions
        self.parsing_table = {}

    def compute_first(self, symbol):
        # Initialize an empty set for the FIRST set of the given symbol.
        first_set = set()

        # If the symbol is a terminal, its FIRST set is just itself.
        if symbol in self.terminals:
            first_set.add(symbol)
        # If the symbol is ε (empty string), add it to the FIRST set.
        elif symbol == "":
            first_set.add(symbol)
        # If the symbol is a non-terminal, compute its FIRST set based on its production rules.
        elif symbol in self.non_terminals:
            for production in self.productions[symbol]:
                # Iterate through the symbols in the production.
                for production_symbol in production:
                    # Compute the FIRST set of the current production symbol.
                    first_of_production_symbol = self.compute_first(production_symbol)

                    # Add all symbols from the FIRST set of the current production symbol
                    # to the FIRST set of the current non-terminal symbol.
                    first_set.update(first_of_production_symbol)

                    # If ε is not in the FIRST set of the current production symbol,
                    # stop processing further symbols in this production.
                    if "" not in first_of_production_symbol:
                        break

        # Return the computed FIRST set for the given symbol.
        return first_set

    def compute_follow(self, symbol):
        # Initialize an empty set for the FOLLOW set of the given symbol.
        follow_set = set()

        # Add the START symbol's FOLLOW set to the FOLLOW set of the start symbol.
        if symbol == self.start_symbol:
            follow_set.add("$")

        # Iterate through the non-terminals to compute FOLLOW sets.
        for non_terminal in self.non_terminals:
            for production in self.productions[non_terminal]:
                # Find the position of the given symbol in the production.
                for i, production_symbol in enumerate(production):
                    if production_symbol == symbol:
                        # Check if there is a symbol after the given symbol in the production.
                        if i + 1 < len(production):
                            next_symbol = production[i + 1]
                            # Compute the FIRST set of the symbols after the given symbol.
                            first_of_next_symbol = self.compute_first(next_symbol)
                            # Add non-empty elements from the FIRST set to the FOLLOW set.
                            follow_set.update(filter(lambda x: x != "", first_of_next_symbol))
                        # If the given symbol is at the end of a production or if ε is in
                        # the FIRST set of symbols after it, add the FOLLOW set of the
                        # non-terminal containing this production to the FOLLOW set.
                        if i == len(production) - 1 or "" in self.compute_first(production[i + 1]):
                            follow_set.update(self.compute_follow(non_terminal))

        # Return the computed FOLLOW set for the given symbol.
        return follow_set

    def construct_parsing_table(self):
        for non_terminal in self.non_terminals:
            for production in self.productions[non_terminal]:
                first_set = self.compute_first(production)
                for terminal in first_set:
                    self.parsing_table[(non_terminal, terminal)] = production
                if "" in first_set or production == "":
                    follow_set = self.compute_follow(non_terminal)
                    for terminal in follow_set:
                        self.parsing_table[(non_terminal, terminal)] = production

    def print_parsing_table(self):
        print("LL(1) Parsing Table:")
        print("{:<15} {:<15} {:<15}".format("Non-Terminal", "Terminal", "Production"))
        for key, value in self.parsing_table.items():
            non_terminal, terminal = key
            production = value
            print("{:<15} {:<15} {:<15}".format(non_terminal, terminal, production))

# Example usage
non_terminals = {'S', 'A', 'B'}
terminals = {'a', 'b'}
start_symbol = 'S'
productions = {
    'S': ['aAB', 'b'],
    'A': ['a', ''],
    'B': ['b', '']
}

ll1_parser = LL1ParsingTable(non_terminals, terminals, start_symbol, productions)
ll1_parser.construct_parsing_table()
ll1_parser.print_parsing_table()


LL(1) Parsing Table:
Non-Terminal    Terminal        Production     
A               a               a              
A                                              
A               b                              
B               b               b              
B                                              
B               $                              
S               b               b              
